In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity


using StatsBase
using Combinatorics
using Plots


# parameters
# parameters

n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)

 Activating environment at `~/hypergraph_modularities_code/Project.toml`
┌ Info: Precompiling HypergraphModularity [0c934d27-dd44-49d7-950f-bd4be7819e54]
└ @ Base loading.jl:1260
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fata

(::HypergraphModularity.var"#Ω#8"{HypergraphModularity.var"#Ω#3#9"{typeof(ω),Int64}}) (generic function with 1 method)

In [3]:
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

HypergraphModularity.hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [3, 2, 3, 4, 3, 1, 2, 3, 2, 2  …  1, 2, 1, 3, 3, 1, 3, 1, 2, 4]


In [5]:
## Create a random clustering with k clusters

k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,Ω;α=α0)

-761.6375316443888

In [8]:
@time T1 = first_term_eval(H,c,Ω;α=α0)
@time Hyp, w = hyperedge_formatting(H)
@time T2 = first_term_v2(Hyp,w,c,Ω;α=α0)

T1 ≈ T2

  0.000138 seconds (1.30 k allocations: 67.078 KiB)
  0.000011 seconds (21 allocations: 6.062 KiB)
  0.000113 seconds (1.30 k allocations: 65.734 KiB)


true